In [66]:
import requests 
import json
import pandas as pd
import numpy as np
from numpy import array
from datetime import datetime, timedelta
import mysql.connector
import alc_connect as alc
import config


## Function To Get Brooklyn Weather For Days Starting 2019

In [2]:
def list_of_days():
    days_list = []
    starting_x = 1546214400
    while starting_x < 1572480000:
        starting_x = starting_x + 86400
        
        days_list.append(starting_x)
    return days_list

In [6]:
ut = list_of_days()

In [7]:
def create_link():
    base = 'https://api.darksky.net/forecast/'+config.api_sky+'/40.6782,-73.9442,'
    end = '?exclude=currently,minutely,hourly,alerts,flags'
    list_of_links = []
    for i in ut:
        list_of_links.append(base + str(i) + end)
    return list_of_links

In [8]:
links_list = create_link()

In [9]:
len(links_list)

304

In [10]:
def create_weather():
    weather_data = [] 
    for i in links_list:
        x = requests.get(i)
        weather_data.append(x.content)
    return weather_data

In [13]:
weather_data_list = create_weather()

In [14]:
weather_data = weather_data_list

In [15]:
weather_data

[b'{"latitude":40.6782,"longitude":-73.9442,"timezone":"America/New_York","daily":{"data":[{"time":1546232400,"summary":"Rain starting in the afternoon.","icon":"rain","sunriseTime":1546258860,"sunsetTime":1546292340,"moonPhase":0.84,"precipIntensity":0.0292,"precipIntensityMax":0.1909,"precipIntensityMaxTime":1546311600,"precipProbability":1,"precipType":"rain","temperatureHigh":44.68,"temperatureHighTime":1546281480,"temperatureLow":42.88,"temperatureLowTime":1546300800,"apparentTemperatureHigh":39.72,"apparentTemperatureHighTime":1546279200,"apparentTemperatureLow":38.15,"apparentTemperatureLowTime":1546301340,"dewPoint":36.07,"humidity":0.89,"pressure":1024.3,"windSpeed":5.1,"windGust":10.61,"windGustTime":1546311240,"windBearing":139,"cloudCover":0.67,"uvIndex":2,"uvIndexTime":1546275600,"visibility":8.378,"ozone":266.5,"temperatureMin":31.43,"temperatureMinTime":1546260480,"temperatureMax":46.57,"temperatureMaxTime":1546315200,"apparentTemperatureMin":31.92,"apparentTemperatureMi

## Parsing Weather Data

In [16]:
new_list = []
for i in weather_data_list:
    p = json.loads(i)
    x = {}
    for k,v in p.items():
        if k == 'daily':
            x['daily'] = v
    new_list.append(x)

In [17]:
len(new_list)

304

In [18]:
cleaner_weather = []
for i in new_list:
    for k,v in i.items():
        x = v
        cleaner_weather.append(x)


In [19]:
cleaner_weather1 = []
for i in cleaner_weather:
    for k,v in i.items():
        x = v
        cleaner_weather1.append(x)


In [20]:
cw = []
for i in cleaner_weather1:
    for x in i: 
        final_weather_list = {}
        for k,v in x.items():
            if k == 'time':
                final_weather_list['time'] = v
            if k == 'icon':
                final_weather_list['icon'] = v
            if k == 'moonPhase':
                final_weather_list['moonPhase'] = v
            if k == 'precipIntensity':
                final_weather_list['precipIntensity'] = v
            if k == 'apparentTemperatureHigh':
                final_weather_list['apparentTemperatureHigh'] = v
            if k == 'apparentTemperatureLow':
                final_weather_list['apparentTemperatureLow'] = v
            if k == 'windSpeed':
                final_weather_list['windSpeed'] = v
        cw.append(final_weather_list)
        
dfcw = pd.DataFrame(cw)
dfcw.head()

,time,icon,moonPhase,precipIntensity,apparentTemperatureHigh,apparentTemperatureLow,windSpeed
0,1546232400,rain,0.84,0.0292,39.72,38.15,5.10
1,1546318800,rain,0.87,0.0060,57.09,29.09,10.93
2,1546405200,partly-cloudy-day,0.91,0.0000,37.41,31.37,5.73
3,1546491600,partly-cloudy-day,0.94,0.0000,39.41,27.30,6.17
4,1546578000,clear-day,0.97,0.0000,41.33,38.02,6.61


In [21]:
dfcw['time'] = pd.to_datetime(dfcw['time'], unit='s')

In [24]:
dfcw.drop([0], inplace=True)

In [25]:
dfcw['Average_Temp'] = dfcw[['apparentTemperatureHigh', 'apparentTemperatureLow']].apply(lambda x: (x['apparentTemperatureHigh'] + x['apparentTemperatureLow'])/2, axis = 1)

In [26]:
dfcw.drop(['apparentTemperatureHigh', 'apparentTemperatureLow'], axis = 1, inplace=True)

In [53]:
dfcw.drop(['Precipitation'], axis = 1, inplace=True)

In [28]:
dfcw = dfcw.dropna(how='any', axis=0)

In [29]:
dfcw['time'] = pd.to_datetime(dfcw['time']).dt.normalize()

In [31]:
dfcw.rename(columns={'time': 'Date', 'icon': "Forecast", 'moonPhase': 'Moon Phase', 'precipIntensity': 'Precipitation', 'windSpeed': 'Wind Speed'}, inplace=True)

In [61]:
dfcw['Average_Temp'] = dfcw['Average_Temp'].round()

In [63]:
dfcw

,Date,Forecast,Moon Phase,Wind Speed,Average_Temp
1,2019-01-01,rain,0.87,10.93,43.0
2,2019-01-02,partly-cloudy-day,0.91,5.73,34.0
3,2019-01-03,partly-cloudy-day,0.94,6.17,33.0
4,2019-01-04,clear-day,0.97,6.61,40.0
5,2019-01-05,rain,0.00,8.03,37.0
...,...,...,...,...,...
299,2019-10-26,clear-day,0.96,4.61,60.0
300,2019-10-27,rain,1.00,8.87,62.0
301,2019-10-28,partly-cloudy-day,0.03,5.09,58.0
302,2019-10-29,rain,0.07,5.29,59.0


moon phase -- a value of 0 corresponds to a new moon, 0.25 to a first quarter moon, 0.5 to a full moon, and 0.75 to a last quarter moon.

## Sending to SQL

In [67]:
dfcw.to_sql('brooklyn_weather', alc.db, if_exists = 'replace', index = False)